# Simulation Settings

In [ ]:
import numpy as np
from cideMOD import (
    CellParser,
    ErrorCheck,
    NDProblem,
    Problem,
    SolverCrashed,
    Trigger,
    init_results_folder,
    ModelOptions,
)

In [ ]:
overwrite = False
case = "Xu_2015"
data_path = f"../../data/data_{case}"
params = "params.json"

model_options = ModelOptions(mode='P4D', solve_thermal = True)

C_rate = 1 # Positive == discharge

T_0 = 273 + 20 # Initial cell temperature [K]

v_min = 2.1 # Cut-off voltage

timestep = 5 # int, float

H = 297e-3 # New cell height [m]

W = 210e-3 # New cell width [m]

h_c = 660 # Effective cooler-cell thermal contact conductance [W/m^2 K]

T_cooler = 273 + 20 # Cooler temperature [K]


### Resize the default cell

In [ ]:
import os
import json

with open(os.path.join(data_path, params),'r') as fin:
    params_dict = json.load(fin)

for key in ['negativeElectrode', 'positiveElectrode', 'separator', 'negativeCurrentCollector', 'positiveCurrentCollector']:
    if key not in params_dict:
        continue
    params_dict[key]['height']['value'] = H
    params_dict[key]['width']['value'] = W

cell = CellParser(params_dict, data_path=data_path)

# Cooler Case

In [ ]:
save_path = init_results_folder(
    f"{case}_{C_rate:.3g}C_cooler_case", overwrite=overwrite, copy_files=[params_dict, model_options.dict()], filenames=['params', 'simulation_options']
)

problem = NDProblem(cell, model_options, save_path=save_path)
problem.set_cell_state(1, T_0, T_0)
problem.add_thermal_boundary_condition('Y_m', h_t = h_c, T_ref = T_cooler )
problem.setup()
I_app = -C_rate * problem.Q
t_f = 3600 /abs(C_rate)*1.25

status = problem.solve_ie( min_step=timestep, i_app=I_app, t_f=t_f, store_delay=1, adaptive=True, triggers=[Trigger(v_min, "v")] )

# Adiabatic Case

In [ ]:
save_path = init_results_folder(
    f"{case}_{C_rate:.3g}C_adiabatic_case", overwrite=overwrite, copy_files=[params_dict, model_options.dict()], filenames=['params', 'simulation_options']
)

problem = NDProblem(cell, model_options, save_path=save_path)
problem.set_cell_state(1, T_0, T_0)
problem.setup()
I_app = -C_rate * problem.Q
t_f = 3600 /abs(C_rate)*1.25

status = problem.solve_ie( min_step=timestep, i_app=I_app, t_f=t_f, store_delay=1, adaptive=True, triggers=[Trigger(v_min, "v")] )